In [64]:
# !pip install kor
# !pip install markdownify

In [65]:
import openai,os,requests,json,bs4
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [66]:
# Kor!
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
# LangChain Models
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
# Standard Helpers
import pandas as pd
import requests
import time
import json
from datetime import datetime
# Text Helpers
from bs4 import BeautifulSoup
from markdownify import markdownify as md
# For token counting
from langchain.callbacks import get_openai_callback
def printOutput(output):
    print(json.dumps(output,sort_keys=True, indent=3))

In [67]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", # Cheaper but less reliable
    # model_name="gpt-4",
    temperature=0,
    max_tokens=2048,
)

In [68]:
ingredients_schema = Object(
    id='ingredients',
    description='Ingredient of the recipe',
    attributes=[
        Text(id='ingredient_name',description='材料名'),
        Text(id='amount',description='材料の分量'),
    ],
    many=True
)

instruction_schema = Object(
    id='instruction',
    description='Instruction of the recipe',
    attributes=[
        Number(id='step',description='手順番号'),
        Text(id='description',description='指示'),
    ],
    many=True
)

recipe_schema = Object(
    id='recipe',
    description='Recipe data',
    attributes=[
        Text(id='name',description='レシピ名'),
        Text(id='description',description='レシピのおすすめ文言'),
        # Text(id='appliances',description='使用機器',many=True),
        # Text(id='tags',description='レシピ特徴タグ',many=True),
        # ingredients_schema,
        # instruction_schema
    ]
)

In [69]:
url = 'https://park.ajinomoto.co.jp/recipe/card/705645/'
response = requests.get(url)

In [70]:
soup = BeautifulSoup(response.text, 'html.parser')
text = soup.get_text()

In [71]:
chain = create_extraction_chain(llm, recipe_schema, input_formatter="triple_quotes")

In [72]:
output = chain.predict_and_parse(text=text)["data"]
printOutput(output)

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4503 tokens. Please reduce the length of the messages.